# Create a Slurm File to Lotus Queue

This notebook makes a slurm file that can be submitted to the lotus queue to compute the profile in a specified box.

In [1]:
import os
import glob

In [2]:
# Define box to compute:
box_name = 'amv'
var      = 'votemper'
region   = 'atlmsk'
imin     = '-75'
imax     = '-7.5'
jmin     = '0'
jmax     = '60'

# Make list of available ensemble members:
datadir = '/gws/nopw/j04/canari/shared/large-ensemble/priority/HIST2/'
outputs = glob.glob(datadir + '*')
ENSs = []
for output in outputs:
    ENSs.append(output.split('/')[-1])

In [3]:
# Make array of ensemble members and file names:
ens      = ('(')
outfiles = ('(')
for ENS in ENSs:
    if ens == '(':
        ens      = (ens      + '"'+ ENS + '"')
        outfiles = (outfiles + '"'+ box_name + '_' + var + '_' + ENS + '.nc' + '"')
    else:
        ens = (ens + ' "' + ENS + '"')
        outfiles = (outfiles + ' "'+ box_name + '_' + var + '_' + ENS + '.nc' + '"')

ns = len(ENSs)

# Make slurm file:
array_short = (box_name + '.slurm')
line1 = '#!/bin/bash \n'
line2 = '#SBATCH --account=short4hr \n'
line3 = '#SBATCH -p short-serial-4hr \n'
line4 = '#SBATCH -o %J.out \n'
line5 = '#SBATCH -e %J.err \n'
line6 = '#SBATCH --time 1:00:00 \n'
line7 = '#SBATCH --mem=32000 \n'
line8 = ('#SBATCH --array=0-' + str(ns-1))
line9 = '\n'
line10= 'ENS=' + ens      + ') \n'
line11= 'OUT=' + outfiles + ') \n'
line12= 'conda activate canari-sprint \n'
line13= ('python box_profile.py ' + ' ${ENS[$SLURM_ARRAY_TASK_ID]} ' + var + ' ' + region + ' ' + imin + ' ' + imax + ' ' + jmin + ' ' + jmax + ' ${OUT[$SLURM_ARRAY_TASK_ID]}')
file = open(array_short,'w') 
file.writelines([line1, line2, line3, line4, line5, line6, line7, line8, line9, line10, line11, line12, line13])
file.close()